# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** First steps with data

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
import itertools
from collections import OrderedDict
import os
import sys

2023-06-12 13:30:09.424062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Initializing test set
np.random.seed(3)
url = 'https://www.dropbox.com/s/hv4uau8q4wwg00k/final_data.csv?dl=1'
data_org = pd.read_csv(url)
test_data = data_org.sample(frac=0.1)
train_data = data_org.drop(test_data.index)

data_org

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

In [3]:
sum_vec = data_org.iloc[:, :9].sum(axis=1)
data2 = data_org.iloc[:, :9].divide(sum_vec, axis='index')
data = pd.concat([data2, data_org['seq']], axis=1)
data

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,ACCTTGTATTAGGTATTTATTTCCACAAAAGTTTGATGCTTACAAC...
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,TGTGCACAGAAGTTATATACATATATGGGTATATCTATGTAACAAA...
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,TACTTTATGCAAAAAAAAATATACATTTATTTATAGGTCTCAATAC...
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,TGACTTTCAAACCATTTTAATATTTCAAATATTCCAGAACAATCCC...
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,AACCCGCTCGGGTCCCCTTCCACACTGTGGAAGCTTTGTTCTTTCG...
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,GGGAAGAAAGGAGCCTGACTCTTATGATGGAATAACCACAAATCAG...
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,GTGTCGGACGGCATGACAGGCAGCAATCCTGTGTCCCCTGCCTCAT...
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,ATGGCGGGGACCTCCGCGCCAGGCAGCAAGAGGCGGAGCGAGCCCC...
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,CTACTGGAACGCCCCCCTCAATCTAGCCTCCCCCACATAACTCTCT...


In [12]:
# One hot encode the 'seq' attribute of the above table
mapping = { 
            'A': 0,
            'C': 1,
            'G': 2,
            'T': 3
            }
one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
data_one_hot_karas = data['seq'].apply(one_hot_encode_lam)

data_one_hot_karas

# Now just injecting this modified 'seq' back into the pandas frame 
data_new_karas =  pd.concat([data.iloc[:, :9], data_one_hot_karas], axis=1)

data_new_karas

,ERM,KDEL,LMA,MITO,NES,NIK,NLS,NUCP,OMM,seq
0,0.207338,0.128871,0.079991,0.044906,0.082833,0.088109,0.061681,0.106670,0.199601,"[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [..."
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.922605,0.077395,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
2,0.064346,0.125935,0.108187,0.082002,0.091019,0.160823,0.142649,0.131566,0.093474,"[[0.0, 0.0, 0.0, 1.0], [1.0, 0.0, 0.0, 0.0], [..."
3,0.141112,0.094932,0.211394,0.028049,0.102672,0.102581,0.124625,0.090767,0.103868,"[[0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0], [..."
4,0.172315,0.115240,0.132014,0.039526,0.115743,0.088187,0.142063,0.105034,0.089878,"[[1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [..."
...,...,...,...,...,...,...,...,...,...,...
13805,0.000000,0.000000,0.734451,0.109981,0.073566,0.000000,0.082002,0.000000,0.000000,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 0.0], [..."
13806,0.067811,0.056029,0.110082,0.010354,0.050838,0.430809,0.110393,0.000000,0.163685,"[[0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13807,0.006797,0.081510,0.077941,0.046627,0.059330,0.330415,0.263028,0.045733,0.088620,"[[1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."
13808,0.000000,0.077421,0.315861,0.053726,0.132350,0.051426,0.189963,0.110591,0.068661,"[[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [..."


In [ ]:
# Splitting for 5fold

folds_total = 5

kf = KFold(n_splits=10, shuffle=True, random_state=1234)
folds = kf.split(X, y)

# folds now contains a list of lists. Each sublist contains all the indices for the pandas data entries to be used in the respective fold

In [21]:
# TODO: what is this?

template = [0] * 24  # dim([a,c,g,t]) * dim([f,t,i,h,m,s])
combined_encoding = OrderedDict()
combined_encoding['UNK'] = template
for i, (key_seq, key_ann) in enumerate(
        itertools.product(['A', 'C', 'G', 'T', 'N'], ['F', 'T', 'I', 'H', 'M', 'S'])):
    tmp = template.copy()
    if key_seq == 'N':
        for n in ['A', 'C', 'G', 'T']:
            tmp[np.nonzero(combined_encoding[n + key_ann])[0][0]] = 0.25
        combined_encoding[key_seq + key_ann] = tmp
    else:
        tmp[i] = 1  # normal one-hot encoding as it is...
        combined_encoding[key_seq + key_ann] = tmp
encoding_keys = list(combined_encoding.keys())
encoding_vectors = np.array(list(combined_encoding.values()))
encoding_vectors

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       

### **2** |  Importing models...

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [17]:
# Import NN
from RNAtracker import RNATracker

# TODO: do a normal CNN


# Set paths for model output
try:
    os.mkdir('~/Downloads/model_outputs/')
except Exception as e:
    print("Output folder already exists")

model_output_folder = '~/Downloads/model_outputs'



In [ ]:

# TODO: with understand what we have to predict, we can allocate X and y
# Also: the kwargsvalues are hyperparameters of which we will select default values from the RNAtracker repo

for i, (train_indices, test_indices) in enumerate(folds):
    print('Evaluating KFolds {}/{}'.format(str(i + 1), str(folds_total)))
    model = RNATracker(max_len, nb_classes, model_output_folder, kfold_index=i)
    #model.build_model(nb_filters=kwargs['nb_filters'], filters_length=kwargs['filters_length'],
    #                          pooling_size=kwargs['pooling_size'], lstm_units=kwargs['lstm_units'],
    #                          embedding_vec=encoding_vectors)
    
    model.build_model_advanced_masking(nb_filters=kwargs['nb_filters'],
                                                   filters_length=kwargs['filters_length'],
                                                   pooling_size=kwargs['pooling_size'],
                                                   lstm_units=kwargs['lstm_units'],
                                                   embedding_vec=encoding_vectors)
    

    model.train(X[train_indices], y[train_indices], batch_size, kwargs['epochs'])

    model.evaluate(X[test_indices], y[test_indices], dataset)

    K.clear_session()





